In [1]:
from collections import namedtuple
from pathlib import Path

import numpy as np

WORKDIR = Path.cwd().absolute()
INPUTFILE = WORKDIR / "2022-17.txt"

with INPUTFILE.open("r") as file:
    inputs = [line.strip() for line in file.readlines()]

test = [">>><<><>><<<>><>>><<<>>><<<><<<>><>><<>>"]
# inputs = test

# inputs[0:8]

In [2]:
"""
####

.#.
###
.#.

..#
..#
###

#
#
#
#

##
##
"""

rocks = [
    (np.array([[1,1,1,1], [0,0,0,0], [0,0,0,0], [0,0,0,0]], dtype=np.uint8), 1),
    (np.array([[0,1,0,0], [1,1,1,0], [0,1,0,0], [0,0,0,0]], dtype=np.uint8), 3),
    (np.array([[1,1,1,0], [0,0,1,0], [0,0,1,0], [0,0,0,0]], dtype=np.uint8), 3),
    (np.array([[1,0,0,0], [1,0,0,0], [1,0,0,0], [1,0,0,0]], dtype=np.uint8), 4),
    (np.array([[1,1,0,0], [1,1,0,0], [0,0,0,0], [0,0,0,0]], dtype=np.uint8), 2)
]


In [3]:
# 2022 pieces (round up to 2048) x 4 blocks high
space = np.zeros((16384,12), dtype=np.uint8)    # width = left wall + 7 spaces + right wall + "shape overlap"

space[0,:] = 1  # floor
space[:,0] = 1  # left side
space[:,8] = 1  # right side

# print(space[-1,:])
# print(space[-2,:])
# print("...")
# print(space[1,:])
# print(space[0,:])

gusts = inputs[0]
start = 4   # floor + 3

gust = 0
for rock in range(2022):

    shape, height = rocks[rock % len(rocks)]
    x, y = 3, start

    try:

        drop = True
        while drop:

            # air gust
            if gusts[gust % len(gusts)] == ">":
                test = x + 1
            else:
                test = x - 1
            if not np.any(space[y:y+4,test:test+4] & shape):
                x = test
            gust += 1

            # gravity
            test = y - 1
            if not np.any(space[test:test+4,x:x+4] & shape):
                y = test
            else:
                drop = False

    except Exception as e:
        print(f"{space.shape=}")
        print(f"{start=}")
        print(f"{rock=}")
        print(f"{shape=}")
        print(f"{height=}")
        print(f"{x=}, {y=}, {test=}")
        raise e

    space[y:y+4,x:x+4] |= shape
    start = max(start, y + height + 3)

apex = start - 3

print(f"Part 1: height is {apex-1}")


Part 1: height is 3144


In [12]:
from hashlib import md5, sha256
from functools import reduce

# 2022 pieces (round up to 2048) x 4 blocks high
space = np.zeros((1_048_576,12), dtype=np.uint8)

space[0,:] = 1  # floor
space[:,0] = 1  # left side
space[:,8] = 1  # right side

gusts = inputs[0]
start = 1   # floor + 3

# history = {index:[] for index in range(5)}
history = []

num_rocks = len(rocks)
gust = 0
num_gusts = len(gusts)
for rock in range(1_048_576):

    irock = rock % num_rocks
    shape, height = rocks[irock]
    x, y = 3, start+3

    try:

        if (gust == 0) and (irock == 0):
            coverage = np.zeros(7, dtype=np.uint8)
            testy = start - 1
            hashed = md5()
            while not np.all(coverage):
                hashed.update(reduce(lambda x, y: x+(y[1]<<y[0]), enumerate(space[testy,1:8]), 0))
                coverage |= space[testy,1:8]
                testy -= 1
            digest = hashed.hexdigest()
            if digest in history:
                print(f"Found matching state for rock {rock} at {history.index(digest)}")
                break
            history.append(digest)

        # # Before dropping the rock, record the "history" of the rocks below
        # coverage = np.zeros(7, dtype=np.uint8)
        # testy = start - 1
        # hashed = md5()
        # while not np.all(coverage):
        #     hashed.update(reduce(lambda x, y: x+(y[1]<<y[0]), enumerate(space[testy,1:8]), 0))
        #     coverage |= space[testy,1:8]
        #     testy -= 1
        # digest = hashed.hexdigest()
        # if digest in history[irock]:
        #     print(f"Found matching state for rock {rock}({irock}) at {history[irock].index(digest)}")
        #     for _ in range(start - 1,-1,-1):
        #         print(f"{rock:3}/{_:3}: {space[_,1:8]}")
        #     break
        # history[irock].append(digest)

        drop = True
        while drop:

            # air gust
            if gusts[gust] == ">":
                test = x + 1
            else:
                test = x - 1
            if not np.any(space[y:y+4,test:test+4] & shape):
                x = test
            gust += 1
            gust %= num_gusts

            # gravity
            test = y - 1
            if not np.any(space[test:test+4,x:x+4] & shape):
                y = test
            else:
                drop = False

    except Exception as e:
        print(f"{space.shape=}")
        print(f"{start=}")
        print(f"{rock=}")
        print(f"{shape=}")
        print(f"{height=}")
        print(f"{x=}, {y=}, {test=}")
        raise e

    space[y:y+4,x:x+4] |= shape
    start = max(start, y + height)

    if ((rock % 5) == 4) and np.all(space[start-1,1:8]):
        print(f"Found new 'floor' after dropping rock {rock}")
        break
else:
    print("Never found a new 'floor'.")

# print(f"Part 1: height is {apex-1}")


space.shape=(1048576, 12)
start=1048571
rock=669939
shape=array([[1, 1, 0, 0],
       [1, 1, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0]], dtype=uint8)
height=2
x=3, y=1048574, test=4


ValueError: operands could not be broadcast together with shapes (2,4) (4,4) 

In [34]:
np.all(space[0,1:8])

True